In [2]:
#import scrublet as scr
import numpy as np
import pandas as pd
import statistics as st
import re
import csv
import scanpy as sc 
import scanpy.external as sce
import phate
import matplotlib
from matplotlib import pyplot as plt
import warnings
from scipy.stats import binom
from scipy.stats import multinomial
import seaborn
from scipy.stats import hypergeom
import warnings
warnings.filterwarnings('ignore')
import scrublet as scr
import harmonypy as hm

def adjust_feature(feature_matrix, feature_id):
    c, f = feature_matrix.shape
    feature_matrix = feature_matrix.copy().todense()
    
    for i in np.arange(f):
        feature_umi_array = np.asarray(feature_matrix[:,i]).squeeze()

        feature_umi_array_sort = feature_umi_array[np.argsort(feature_umi_array * -1)]
        feature_cumsum = np.cumsum(feature_umi_array_sort)

        cell_num = np.sum(feature_umi_array_sort > 0)

        turn_point = np.sum(np.diff(feature_cumsum) / np.sum(feature_umi_array) > (1 / cell_num))
        feature_matrix[np.where(feature_umi_array < feature_umi_array_sort[turn_point]), i] = 0
        
    return feature_matrix

In [3]:
adata = sc.read_h5ad('./DAP3_SC_072821.h5ad')

In [4]:
SG_DF = pd.read_pickle('./DAP3_SGRNA_IND_072821.pkl')

In [5]:
SG_DF_Grouped = pd.read_pickle('./DAP3_SGRNA_GROUP_072821.pkl')

In [8]:
adata = adata[adata.obs.louvain != '7']

In [13]:
sg_pos = []
for cell in adata.obs.index:
    try:
        if sum(SG_DF.loc[cell] > 0) > 0:
            sg_pos.append(cell)
    except:
        continue
SG_DF = SG_DF.loc[sg_pos]
SG_DF_Grouped = SG_DF_Grouped.loc[sg_pos]
adata = adata[sg_pos]

In [17]:
sc.tl.rank_genes_groups(adata, 'louvain', method='wilcoxon')

Trying to set attribute `.uns` of view, copying.


In [18]:
adata.write('./DAP3_SC_072821.h5ad')

In [15]:
SG_DF.to_pickle('./DAP3_SGRNA_IND_SC_072821.pkl')

In [16]:
SG_DF_Grouped.to_pickle('./DAP3_SGRNA_GROUP_SC_072821.pkl')